In [3]:
import pandas as pd
from konlpy.tag import Okt # komoran, han, kkma
from jamo import h2j, j2hcj

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Dropout

import numpy as np

In [4]:
# 각자 경로에 맞게 수정해주세요
train = pd.read_csv(r'data/train_merge.csv')
test = pd.read_csv(r'data/test_merge.csv')

# 자모음 쪼갠 텍스트를 한글자씩 리스트로 만들어서 데이터셋 만든다
def textToList(text):
    li = []
    for t in text:
        li.append(list(t))
    return li

train['text'] = train['text'].apply(lambda x : j2hcj(h2j(x)))
test['text'] = test['text'].apply(lambda x : j2hcj(h2j(x)))

X_train = textToList(train['text'])
y_train = train['immoral']

X_test = textToList(test['text'])
y_test = test['immoral']

tokenizer = Tokenizer(num_words=110)
tokenizer.fit_on_texts(X_train) # 단어 인덱스 구축
text_sequences = tokenizer.texts_to_sequences(X_train) # 문자열 -> 인덱스 리스트
                                                            # '나는 천재다 나는 멋있다' -> [1, 2, 1, 3]
# train data로 fit 시켜준 tokenizer로 test data도 인덱스로 변경
text_sequences_test = tokenizer.texts_to_sequences(X_test)

word_vocab = tokenizer.word_index # 딕셔너리 형태
print("전체 단어 개수: ", len(word_vocab)) # 전체 단어 개수 확인

MAX_SEQUENCE_LENGTH = 100 # 문장 최대 길이

X_train = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='pre')
y_train = np.array(y_train) # 각 리뷰의 감정을 넘파이 배열로 만든다.

X_test = pad_sequences(text_sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre')
y_test = np.array(y_test) # 각 리뷰의 감정을 넘파이 배열로 만든다.

전체 단어 개수:  166


In [ ]:
## 아래 변수 원하는대로 수정
embedding_dim = 100 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수
####
# 모델링 시작
## Conv1D 갯수, num_filters 원하는대로 수정
model = Sequential()
model.add(Embedding(len(word_vocab)+1, 100))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
# Pooling 없애도 괜찮습니다
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics=['accuracy'])
model.summary()

early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, epochs= 100, batch_size = 100, validation_split=0.2, callbacks=[early])


model.save('convModel.h5')